IMPORTING NECESSARY MODULES

In [7]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
import seaborn as sns
import matplotlib.pyplot as plt

/kaggle/input/machine-learning-101/ML101_train_dataset.csvReading csv using pandas files and preprocessing the given data to remove nan values and inconsistent(negative) values.

In [8]:
df = pd.read_csv('/kaggle/input/machine-learning-101/ML101_train_dataset.csv')

nan_counts = df.isna().sum(axis=1)
total_columns = len(df.columns)-1

threshold = 1/3

df = df.drop(nan_counts[nan_counts>=total_columns*threshold].index,axis=0)
def putNaN(x):
    if x < 0:
        return np.nan
    else:
        return x

numeric_columns = []
for i in df.columns:
    if(i=="Gender" or i=="LifeStyle"):
        continue
    numeric_columns.append(i)


df[numeric_columns] = df[numeric_columns].applymap(putNaN)
df.dropna(subset=['Gender'], how='all', inplace=True)
df.dropna(subset=['LifeStyle'], how='all', inplace=True)

valid_lifestyles = ['Good', 'Bad', 'Average', 'Great']
df = df[df['LifeStyle'].isin(valid_lifestyles)]

df[numeric_columns] = df[numeric_columns].interpolate(method='linear', axis=0)

train_data=df.copy() # preprocessed training data to be used for classification

Code for prediction of blood pressure (systolic and diastolic) using regression

In [9]:

target_variable1,target_variable2 = 'Systolic BP' , 'Diastolic BP'
features = numeric_columns.copy()


features.remove(target_variable1)
features.remove(target_variable2)

categorical_column = ['Gender']
df['Gender'] = df['Gender'].map({'Male': 0, 'Female': 1})
features.append('Gender')
X_train1,X_test1,y_train1,y_test1 = train_test_split(df[features],df[target_variable1],test_size=0.00001,random_state=42)
X_train1 = np.array(X_train1)
X_test1 = np.array(X_test1)
y_train1 = np.array(y_train1)
y_test1 = np.array(y_test1)
def regression_parameters(x,y):
    Z = []

    for i in range(len(x)):
        Z.append(x[i])
    
    w = np.dot((np.dot(np.linalg.inv(np.dot(np.transpose(Z),Z)),np.transpose(Z))),np.transpose(y))

    return w

def multiple_regression(x_train,y_train,x_test):
    w = regression_parameters(x_train,y_train)
    y_predict = []
    for i in range(len(x_test)):
        y_predict.append(np.dot(w,np.transpose(x_test[i])))
    
    return y_predict

X_train2,X_test2,y_train2,y_test2 = train_test_split(df[features],df[target_variable2],test_size=0.00001,random_state=42)
X_train2 = np.array(X_train2)
X_test2 = np.array(X_test2)
y_train2 = np.array(y_train2)
y_test2 = np.array(y_test2)
def regression_parameters(x,y):
    Z = []

    for i in range(len(x)):
        Z.append(x[i])
    
    w = np.dot((np.dot(np.linalg.inv(np.dot(np.transpose(Z),Z)),np.transpose(Z))),np.transpose(y))

    return w

def multiple_regression(x_train,y_train,x_test):
    w = regression_parameters(x_train,y_train)
    y_predict = []
    for i in range(len(x_test)):
        y_predict.append(np.dot(w,np.transpose(x_test[i])))
    
    return y_predict

test_data = pd.read_csv('/kaggle/input/machine-learning-101/ML101_dataset_test_feature.csv')
numeric_columns.remove('Systolic BP')
numeric_columns.remove('Diastolic BP')
test_data[numeric_columns] = test_data[numeric_columns].applymap(putNaN)
test_data.dropna(subset=['Gender'], how='all', inplace=True)
# test_data.dropna(subset=['LifeStyle'], how='all', inplace=True)
test_data['Gender'] = test_data['Gender'].map({'Male': 0, 'Female': 1})

X_test = test_data[features].values

y_predict_systolic = multiple_regression(X_train1, y_train1, X_test)


y_predict_diastolic = multiple_regression(X_train2, y_train2, X_test)

results = pd.DataFrame({'Systolic BP': y_predict_systolic, 'Diastolic BP': y_predict_diastolic})


results.to_csv('predicted_blood_pressure.csv', index=False)

Code for Classification of given test data in terms of lifestyle

In [10]:
attr_name=["Average Daily Steps","Hours of Sleep","Caloric Intake","Age","Gender","Height","Weight","Cholesterol level","Blood Sugar level"]
# train_data=pd.read_csv("preprocessed_data.csv")
train_data=train_data.drop(["Systolic BP","Diastolic BP"], axis=1)

gr_by_gender=train_data.groupby("Gender")
male_data=gr_by_gender.get_group("Male")
female_data=gr_by_gender.get_group("Female")

test_data=pd.read_csv("/kaggle/input/machine-learning-101/ML101_dataset_test_feature.csv")
# test_data=x_test
test_data_wot_gender=test_data.drop(["Gender"], axis=1)

attr_name=["Average Daily Steps","Hours of Sleep","Caloric Intake","Age","Gender","Height","Weight","Cholesterol level","Blood Sugar level"]
n=len(attr_name)
class_name=["Bad","Average","Good","Great"]

# CLASSIFICATION FOR MALE

male_data=male_data.drop(["Gender"], axis=1)
y_male=male_data.iloc[:,8:]
m_data_sz=len(y_male)

# separating each class data

grouped_data = male_data.groupby("LifeStyle")
m_c1=grouped_data.get_group("Bad").iloc[:,:8]
m_c2=grouped_data.get_group("Average").iloc[:,:8]
m_c3=grouped_data.get_group("Good").iloc[:,:8]
m_c4=grouped_data.get_group("Great").iloc[:,:8]

# parameter estimation for each class of male
class_c=[m_c1,m_c2,m_c3,m_c4]
male_mean_estimated_matrix=[]
male_cov_estimated_matrix=[]

for i in range(4):
    c_i=(class_c[i])
    mean_vec=np.array(c_i.mean(axis=0,numeric_only=True))
    male_mean_estimated_matrix.append(mean_vec)
    tmp=np.array(c_i).transpose()
    cov_vec=np.cov(tmp)
    male_cov_estimated_matrix.append(cov_vec)

male_prior_prob_c=[]
male_prior_prob_c.append(len(m_c1)/m_data_sz)
male_prior_prob_c.append(len(m_c2)/m_data_sz)
male_prior_prob_c.append(len(m_c3)/m_data_sz)
male_prior_prob_c.append(len(m_c4)/m_data_sz)

# CLASSIFICATION FOR FEMALE

female_data=female_data.drop(["Gender"], axis=1)
y_female=female_data.iloc[:,8:]
f_data_sz=len(y_female)

# separating each class data

grouped_data = female_data.groupby("LifeStyle")
f_c1=grouped_data.get_group("Bad").iloc[:,:8]
f_c2=grouped_data.get_group("Average").iloc[:,:8]
f_c3=grouped_data.get_group("Good").iloc[:,:8]
f_c4=grouped_data.get_group("Great").iloc[:,:8]

# parameter estimation for each class of male
class_c=[f_c1,f_c2,f_c3,f_c4]
female_mean_estimated_matrix=[]
female_cov_estimated_matrix=[]

for i in range(4):
    c_i=(class_c[i])
    mean_vec=np.array(c_i.mean(axis=0,numeric_only=True))
    female_mean_estimated_matrix.append(mean_vec)
    tmp=np.array(c_i).transpose()
    cov_vec=np.cov(tmp)
    female_cov_estimated_matrix.append(cov_vec)

female_prior_prob_c=[]
female_prior_prob_c.append(len(f_c1)/f_data_sz)
female_prior_prob_c.append(len(f_c2)/f_data_sz)
female_prior_prob_c.append(len(f_c3)/f_data_sz)
female_prior_prob_c.append(len(f_c4)/f_data_sz)

y_test_output=[]

# FUNCTION FOR MAHALNOBIS DISTANCE
def md(x,mu,sig):
    x1=np.array(x-mu)
    x2=np.linalg.inv(sig)
    a=np.matmul(x1.transpose(),x2)
    b=np.matmul(a,x1)
    return b

# FUNCTION FOR CLASSIFICATION OF MALE TEST DATA
def male_classifier(x):
    prob_ci=[]
    for i in range(4):
        prob_ci.append(md(x,male_mean_estimated_matrix[i],male_cov_estimated_matrix[i]))
    ind=np.argmin(prob_ci)
    y_test_output.append(class_name[ind])

# FUNCTION FOR CLASSIFICATION OF FEMALE TEST DATA
def female_classifier(x):
    prob_ci=[]
    for i in range(4):
        prob_ci.append(md(x,male_mean_estimated_matrix[i],male_cov_estimated_matrix[i]))
    ind=np.argmin(prob_ci)
    y_test_output.append(class_name[ind])

for i in range(len(test_data.index)):
    x_wt_g=list(np.array(test_data.iloc[i:i+1,:])[0])
    x=list(np.array(test_data_wot_gender.iloc[i:i+1,:])[0])
    if(x_wt_g[4]=='Male'):
        male_classifier(x)
    else:
        female_classifier(x)

f_results = pd.DataFrame({'LifeStyle': y_test_output})


f_results.to_csv('predicted_lifestyle.csv', index=False)